In [8]:
from operator import itemgetter, add
from itertools import product, chain, cycle, zip_longest
from functools import reduce

import requests

import pandas as pd

In [695]:
URL_TEST = 'https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_1.json?v=timestamp'

r = requests.get(URL_TEST)
r.raise_for_status()

resp = r.json()
resp.keys()

dict_keys(['russia_stat_struct', 'world_stat_struct', 'isolation_stat_struct', 'top_cases_struct', 'all_russia_bars_struct', 'all_world_bars_struct', 'country_comp_struct_v2'])

In [696]:
len(resp['russia_stat_struct']['dates']), len(resp['russia_stat_struct']['data'])

(31, 86)

In [588]:
URL_TEMPLATE = 'https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_{num}.json?v=timestamp'

In [458]:
i = 1

while i <= 14:
    url = URL_TEMPLATE.format(num=i)
    print(url, i)
    i += 1

https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_1.json?v=timestamp 1
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_2.json?v=timestamp 2
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_3.json?v=timestamp 3
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_4.json?v=timestamp 4
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_5.json?v=timestamp 5
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_6.json?v=timestamp 6
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_7.json?v=timestamp 7
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_8.json?v=timestamp 8
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_9.json?v=timestamp 9
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_10.json?v=timestamp 10
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_11.json?v=timestamp 11
https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_12.json?v

In [459]:
i = 12
results = {}

while True:
    url = URL_TEMPLATE.format(num=i)
    r = requests.get(url)
    
    try:
        r.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(i, err)
        break
        
    results[i] = r.json()
    i += 1

14 404 Client Error: Not Found for url: https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_14.json?v=timestamp


In [460]:
for key, data in results.items():
    try:
        print(key, data['russia_stat_struct'].keys())
    except KeyError as err:
        print('Russia is not found', key, err)

12 dict_keys(['dates', 'data'])
Russia is not found 13 'russia_stat_struct'


In [705]:
from itertools import zip_longest

In [9]:
get_cases = itemgetter('cases')
get_cured = itemgetter('cured')
get_deaths = itemgetter('deaths')
get_value = itemgetter('v')

def concat_cols(data):
    return (data[0],) + data[1]

def parse_region_data(data):
    region = data['info']['name']
#     print(data)
#     print(get_cases(data))
    
    if isinstance(get_cases(data)[0], dict):
#         print('New Format')
        counts = zip_longest(map(get_value, get_cases(data)),
                             map(get_value, get_cured(data)),
                             map(get_value, get_deaths(data)), fillvalue=0)
    else:
#         print('Old Format')
        counts = zip_longest(get_cases(data),
                             get_cured(data),
                             get_deaths(data), fillvalue=0)
#     print(list(counts))
        
    parsed_data = product([region], counts)
    parsed_data = map(concat_cols, parsed_data)
    parsed_data = parsed_data
    
    return parsed_data

def parse_russian_data(data):
    russian_data = data['russia_stat_struct']
    
    dates = russian_data['dates']
    region_data = russian_data['data'].values()
    
    data_size = len(region_data)
    all_dates = cycle(dates)
    
    parsed_data = map(parse_region_data, region_data)
    parsed_data = chain.from_iterable(parsed_data)
    
    parsed_data = zip(all_dates, parsed_data)
    parsed_data = map(concat_cols, parsed_data)
    
    return parsed_data

# def combine_results(acc, nxt):
#     return acc | set(nxt)

def combine_results(acc, nxt):
    key = nxt[0]
    values = nxt[1]
    
    result = product([key], values)
    result = map(concat_cols, result)
    
    return acc | set(result)

In [12]:
URL_TEMPLATE = 'https://yastat.net/s3/milab/2020/covid19-stat/data/data_struct_{num}.json?v=timestamp'
URL_TEMPLATE_2 = 'https://yastat.net/s3/milab/2020/covid19-stat/data/deep_history_struct_{num}.json'

In [13]:
i = 1
results = {}

while True:
    print(str(i))
    
    url = URL_TEMPLATE_2.format(num=i)
    r = requests.get(url)
    
    try:
        r.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        break
    
    data = r.json()
        
    try:
        results[i] = tuple(parse_russian_data(data))
    except KeyError as err:
        print('Russian data is not found', err)
    
    i += 1
    
#results

1
2
3
404 Client Error: Not Found for url: https://yastat.net/s3/milab/2020/covid19-stat/data/deep_history_struct_3.json


In [16]:
full_results = reduce(combine_results, results.items(), set())
full_results = sorted(full_results, key=itemgetter(0, 1, 2))
len(full_results)

15996

In [19]:
df = pd.DataFrame.from_records(full_results, columns=['file_num', 'date', 'region', 'infected', 'recovered', 'dead'], index='file_num')
df[-100:]

,date,region,infected,recovered,dead
file_num,,,,,
2,2020-06-01,Тверская область,1650,1087,22
2,2020-06-01,Томская область,716,326,11
2,2020-06-01,Тульская область,3968,1815,36
2,2020-06-01,Тюменская область,1661,818,8
2,2020-06-01,Удмуртская Республика,682,412,12
...,...,...,...,...,...
2,2020-06-02,Чеченская Республика,1264,847,13
2,2020-06-02,Чувашская Республика,3061,1319,14
2,2020-06-02,Чукотский автономный округ,81,56,1


In [20]:
df.groupby(['date', 'region'], as_index=False).last()

,date,region,infected,recovered,dead
0,2020-03-02,Алтайский край,0,0,0
1,2020-03-02,Амурская область,0,0,0
2,2020-03-02,Архангельская область,0,0,0
3,2020-03-02,Астраханская область,0,0,0
4,2020-03-02,Белгородская область,0,0,0
...,...,...,...,...,...
7993,2020-06-02,Чеченская Республика,1264,847,13
7994,2020-06-02,Чувашская Республика,3061,1319,14
7995,2020-06-02,Чукотский автономный округ,81,56,1
7996,2020-06-02,Ямало-Ненецкий автономный округ,2466,1163,10


In [21]:
7998 / 86

93.0

In [ ]:
pd.Series.is_monotonic_increasing

In [845]:
df.to_csv('../data/russia_stats.csv')

In [ ]:
!head